In [46]:
import numpy as np
from time import ctime
import pickle
import numpy.polynomial.polynomial
import pandas as pd
import warnings
import scipy.stats as sc
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.figure_factory as ff
init_notebook_mode(connected=True)
from ortools.sat.python import cp_model

### Pick up functions used to generate dummy data, model, etc for sectorisation

In [2]:
def return_time_horizon_profiles_dict(random_state=None):
    
    time_horizon_profiles_dict = {
        'flat' : 
            {'y_values' : np.array(np.full((10),1)),
             'order' : 4
            },
        'beginning_heavy' : 
            {'y_values' : np.array([1.  , 1.15, 1.45, 1.15, 1.  , 0.95, 0.9 , 0.85, 0.8 , 0.75]),
             'order' : 4
            },
        'end_heavy' : 
            {'y_values' : np.array([0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.15, 1.45, 1.15, 1.  ]),
             'order' : 4
            },
        'beginning_and_end_heavy' : 
            {'y_values' : np.array([1.05, 1.35, 1.05, 0.9 , 0.65, 0.65, 0.9 , 1.05, 1.35, 1.05]),
             'order' : 4
            },
        'middle_heavy' : 
            {'y_values' : np.array([0.7 , 0.85, 1.  , 1.15, 1.3 , 1.3 , 1.15, 1.  , 0.85, 0.7 ]),
             'order' : 4
            },
        'random' : 
            {'y_values' : sc.norm.rvs(1,0.3,10,random_state=random_state).clip(min=0),
             'order' : 6
            }    
    }
    
    return time_horizon_profiles_dict

def get_time_horizon_profiles_by_time_interval(time_horizon_profile, number_time_intervals,random_state=None):
    
    """
    This function is used to return a set of weights used to vary the taskload over the time horizon, 
    using the time_horizon_profiles_dict. The base weighting in the dictionary is fitted to a 
    polynomial and used to generate a weight for each point on the time horizon. 
    """
    
    time_horizon_profiles_dict = return_time_horizon_profiles_dict()
    
    order = time_horizon_profiles_dict[time_horizon_profile]['order']
    x = np.linspace(0,number_time_intervals,10) 
    y = time_horizon_profiles_dict[time_horizon_profile]['y_values']
    ffit = np.polynomial.polynomial.Polynomial.fit(x, y, deg=order)
    time_intervals = np.arange(number_time_intervals)
    out_array = ffit(time_intervals).clip(min=0.1)
    
    return out_array

def generate_taskload_array(taskload_parameters, random_state=None):
    
    """
    Function to generate an taskload array using specified input parameters.
    """
    
    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    job_normal_dist_params_dict = taskload_parameters['job_normal_dist_params_dict']
    job_time_horizon_profiles_dict = taskload_parameters['job_time_horizon_profiles_dict']
    job_zero_taskload_probability_dict = taskload_parameters['job_zero_taskload_probability_dict']
    
    taskload_array = np.zeros((number_jobs, number_time_intervals))
    
    # create an array of random states - necessary to replicate returned data
    random_states = sc.randint.rvs(0,number_jobs*number_time_intervals,
                                   size=(number_jobs, number_time_intervals),
                                   random_state=random_state)
    
    for row in range(number_jobs):
        time_horizon_profile = get_time_horizon_profiles_by_time_interval(job_time_horizon_profiles_dict[row], 
                                                                          number_time_intervals)
        zero_taskload_chance = job_zero_taskload_probability_dict[row]
        for column in range(number_time_intervals):
            mean = job_normal_dist_params_dict[row]['mean'] * time_horizon_profile[column]
            stdev = job_normal_dist_params_dict[row]['stdev'] * time_horizon_profile[column]
            random_state_n = random_states[row, column]
            if sc.uniform.rvs(0,1,random_state=random_state_n) >= zero_taskload_chance:
                taskload_array[row, column] = np.round(sc.norm.rvs(mean, stdev, size=1, 
                                                                   random_state=random_state_n).clip(min=0),0)[0]
    taskload_array = taskload_array.astype(int)
    
    return taskload_array

def generate_random_taskload_parameters(number_time_intervals, number_jobs, 
                                       job_taskload_parameter_generator_parameters_dict, 
                                       zero_taskload_density=0.25, time_horizon_profile=None, 
                                       random_state=None):

    """
    Function to generate a parameter set, randomised to a certain level given the specified inputs.
    """
    
    # generate taskload parameters for each job
    job_taskload_parameter_generator_mean = job_taskload_parameter_generator_parameters_dict['taskload_dist_means_mean']
    job_taskload_parameter_generator_stdev = job_taskload_parameter_generator_parameters_dict['taskload_dist_means_stdev']
    job_taskload_parameter_generator_stdev_ratio = job_taskload_parameter_generator_parameters_dict['taskload_dist_stdev_ratio']
    job_normal_dist_means = np.round(sc.norm.rvs(job_taskload_parameter_generator_mean, 
                                                  job_taskload_parameter_generator_stdev, 
                                                  size=number_jobs, random_state=random_state).clip(min=0),0)
    job_normal_dist_stdevs = np.round(np.multiply(job_normal_dist_means, 
                                                  job_taskload_parameter_generator_stdev_ratio).clip(min=0),0)
    job_normal_dist_params_dict = {i: {'mean': job_normal_dist_means[i], 'stdev': job_normal_dist_stdevs[i]}
                                   for i in range(number_jobs)}
    
    # if a time_horizon_profile is not provided, just use random ones for each job
    # if it is provided, use that for all jobs
    time_horizon_profiles_dict = return_time_horizon_profiles_dict(random_state=random_state)
    if time_horizon_profile is None:
        time_horizon_profiles_keys = sorted(list(time_horizon_profiles_dict.keys()))
        random_indexes = sc.randint.rvs(0,len(time_horizon_profiles_dict),size=number_jobs,random_state=random_state)
        job_time_horizon_profiles_dict = {i: time_horizon_profiles_keys[random_indexes[i]] for i in range(number_jobs)}
    else:
        job_time_horizon_profiles_dict = {i: time_horizon_profile  for i in range(number_jobs)}
        
    # calculate zero taskload probabilty using the zero_taskload_density
    # parameter as a knob to control density
    job_zero_taskload_probability_dict = {k: (lambda v: 0 if zero_taskload_density == 0 
                                              else 1/pow(v['mean'], 1-zero_taskload_density))(v)
                                          for k, v in job_normal_dist_params_dict.items()}
    
    out_parameters = {
        'number_time_intervals' : number_time_intervals,
        'number_jobs' : number_jobs,
        'job_normal_dist_params_dict' : job_normal_dist_params_dict,
        'job_time_horizon_profiles_dict' : job_time_horizon_profiles_dict,
        'job_zero_taskload_probability_dict' : job_zero_taskload_probability_dict,
        'time_horizon_profiles_dict' : time_horizon_profiles_dict,
    }
    
    return out_parameters
        
def generate_agent_availability_array(number_agents, number_time_intervals, agent_availability_time_intervals_dict):
    
    """
    Function to generate an array of booleans where each row is an agent, each column is
    a time period, and each element indicates whether that agent is working in that
    time period.
    """
    
    out_array = np.zeros((number_agents, number_time_intervals))
    
    for k, v in agent_availability_time_intervals_dict.items():
        
        out_array[k, v[0]:v[1]+1] = np.ones(v[1]-v[0]+1)
        
    return out_array

def return_agent_availabilities(taskload_parameters, number_agents, agent_availability_profile, 
                                shift_length=0, random_state=None):
    
    """
    Function to return an agent availabilties dictionary. Currently the two options are:
        1) full - here each agent can work for the whole time horizon
        2) random - here agents starts are randomly distributed between the first
                    time interval and the last one which would allow them to complete
                    a full shift
    More may be added at a later date.
    """
    
    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    
    if agent_availability_profile == 'full':
        
        agent_availability_time_intervals_dict = {i: (0, number_time_intervals-1) 
                                                  for i in range(number_agents)}
        
    elif agent_availability_profile == 'random':
        
        start_times = sc.randint.rvs(0, number_time_intervals-shift_length, size=number_agents, 
                                     random_state=random_state)
        
        agent_availability_time_intervals_dict = {i: (start_times[i], start_times[i]+shift_length-1) 
                                                  for i in range(number_agents)}
        
    else:
        agent_availability_time_intervals_dict = {}
        warnings.warn('Please use supported agent availability profile.')
    
    return agent_availability_time_intervals_dict

def generate_agent_job_endorsement_array(number_agents, number_jobs, agent_job_endorsements_dict):
    
    """
    Function to generate an array of booleans where each row is an agent, each column is
    a job, and each element indicates whether that agent is able to work that job.
    """
    
    out_array = np.zeros((number_agents, number_jobs))
    
    for k, v in agent_job_endorsements_dict.items():
        
        out_array[k, v] = 1
        
    return out_array

def return_agent_endorsements(number_agents, number_jobs, endorsement_density_distribution_dict, random_state=None):
    
    """
    Function to return an agent endorsement dictionary. The ratio of jobs that agents can do is 
    sampled from a uniform distribution for each agent, with the upper and lower bounds given in 
    endorsement_density_distribution_dict.
    """
    
    lower_bound = endorsement_density_distribution_dict['lower_bound']
    upper_bound = endorsement_density_distribution_dict['upper_bound']
    
    endorsement_densities = sc.uniform.rvs(scale=upper_bound-lower_bound,loc=lower_bound
                                          ,size=number_agents,random_state=random_state)
    
    uniform_array = sc.uniform.rvs(size=(number_agents, number_jobs), random_state=random_state)
    
    agent_job_endorsements_dict = {i: [j for j in range(number_jobs) 
                                       if uniform_array[i,j] <= endorsement_densities[i]
                                      ]
                                   for i in range(number_agents)
                                  }
    
    return agent_job_endorsements_dict

def generate_random_agent_parameters(number_agents, taskload_parameters, endorsement_density_distribution_dict,
                                     agent_availability_profile, shift_length=0, random_state=None):
    
    """
    Function to generate a random set of agent parameters.
    """

    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    
    agent_job_endorsements_dict = return_agent_endorsements(number_agents, number_jobs, 
                                                            endorsement_density_distribution_dict, 
                                                            random_state=random_state)
    
    agent_availability_time_intervals_dict = return_agent_availabilities(taskload_parameters, number_agents, 
                                                                         agent_availability_profile, 
                                                                         shift_length=shift_length, 
                                                                         random_state=None)
    
    agent_parameters = {
        'number_agents' : number_agents,
        'agent_availability_time_intervals_dict' : agent_availability_time_intervals_dict,
        'agent_job_endorsements_dict' : agent_job_endorsements_dict
    }
    
    return agent_parameters

def create_taskload_heatmap(taskload_parameters, taskload_array, in_notebook=False,
                            colorscale='Jet',annotate_heatmap=True):
    
    """
    Create a heatmap to visualise the taskload array.
    """
    
    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    
    time_interval_taskloads = taskload_array.sum(axis=0)
    
    
    z_hover = np.empty((number_jobs, number_time_intervals),dtype='object')
    for i in range(number_time_intervals):
        for j in range(number_jobs):
            z_hover[j, i] = 'time interval ' + str(i) + '  sector #' + str(j) \
            + '   taskload=' + str(taskload_array[j, i])
            
    if annotate_heatmap:
        annotation_text = taskload_array
    else:
        annotation_text = np.empty(taskload_array.shape,dtype='str')
        
    trace = ff.create_annotated_heatmap(z=taskload_array
    , x=[i for i in range(number_time_intervals)]
    , y=[i for i in range(number_jobs)]
    , xgap=1, ygap=1
    , annotation_text=annotation_text
    , colorscale=colorscale
    , text=z_hover
    , hoverinfo='text'
    )
    
    trace.layout.update({'title':'Taskload Matrix',
                        'xaxis':go.layout.XAxis(#title='Time Intervals',
                        tickvals = [i for i in range(number_time_intervals)],
                        ticktext = [str(int(i)) for i in time_interval_taskloads],
                        tickfont = {'size':10},
                        tickangle=30,
                        side='top'),                  
                                                
                         
                         'yaxis':go.layout.YAxis(title='Job ID')})
    
    if in_notebook:
        iplot(trace)
    else:
        plot(trace)
        
        
def is_solution_valid(taskload_array, solution, partition_max):

    max_partition_index = int(solution.max())
    
    valid = True
    for ti in range(solution.shape[1]): 
        
        for partition in range(max_partition_index+1):
            
            job_indexes = np.where(solution[:,ti]==partition)[0]
            temp_array = np.zeros(solution.shape[0])
            temp_array[job_indexes] = 1
            job_workload = np.multiply(temp_array, taskload_array[:,ti]).sum()
                                                 
            if job_workload > partition_max:
                warnings.warn('solution not valid for partition ' + str(partition) + ' in time interval ' + str(ti))
                valid = False
        
    return valid

def make_solution_groups_array(model, taskload_parameters):

    number_jobs = taskload_parameters['number_jobs']
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    solution_array = np.zeros((number_jobs, number_time_intervals))
    
    job_time_to_group_dict = {(k2[0], k2[2]): k2[1] for k2, v2 in 
                                    {k: v for k, v in model.p.get_values().items() if v == 1
                                }.items()}
    
    for k, v in job_time_to_group_dict.items():
        
        solution_array[k[0],k[1]] = v
        
    return solution_array


def make_solution_agents_array(model, taskload_parameters):
    
    number_jobs = taskload_parameters['number_jobs']
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    solution_array = np.zeros((number_jobs, number_time_intervals))
    
    job_time_to_group_dict = {(k2[0], k2[2]): k2[1] for k2, v2 in 
                                    {k: v for k, v in model.p.get_values().items() if v == 1
                                }.items()}
    group_time_to_agent_dict = {(k2[1], k2[2]): k2[0] for k2, v2 in 
                                        {k: v for k, v in model.q.get_values().items() if v == 1
                                    }.items()}
    job_time_to_agent_dict = {k: group_time_to_agent_dict[(v, k[1])]  for k, v in job_time_to_group_dict.items()}
    
    for k, v in job_time_to_agent_dict.items():
        
        solution_array[k[0],k[1]] = v
        
    return solution_array


def make_solution_heatmap(model, taskload_parameters, agent_parameters, taskload_array, 
                          agent_heatmap = True, in_notebook=False, colorscale='Jet',
                          annotate_heatmap=True):
    
    number_jobs = taskload_parameters['number_jobs']
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    if agent_heatmap:
        solution_array = make_solution_agents_array(model, taskload_parameters)
    else:
        solution_array = make_solution_groups_array(model, taskload_parameters)
    
    time_interval_taskloads = taskload_array.sum(axis=0)
    
    z_hover = np.empty((number_jobs, number_time_intervals),dtype='object')
    for i in range(number_time_intervals):
        for j in range(number_jobs):
            z_hover[j, i] = 'time interval ' + str(i) + '  job #' + str(j)
            
    if annotate_heatmap:
        annotation_text = taskload_array
    else:
        annotation_text = np.empty(taskload_array.shape,dtype='str')
        
    trace = ff.create_annotated_heatmap(z=solution_array
        , x=[i for i in range(number_time_intervals)]
        , y=[i for i in range(number_jobs)]
        , xgap=1, ygap=1
        , annotation_text=annotation_text
        , colorscale=colorscale
        , text=z_hover
        , hoverinfo='text'
        , showscale=True
    )
    
    trace.layout.update({'title':'Job to Console Solution',
                        'xaxis':go.layout.XAxis(#title='Time Intervals',
                        tickvals = [i for i in range(number_time_intervals)],
                        ticktext = [str(int(i)) for i in time_interval_taskloads],
                        tickfont = {'size':10},
                        tickangle=30,
                        side='top'),                  
                                                
                         
                         'yaxis':go.layout.YAxis(title='Job ID')})
    
    if in_notebook:
        iplot(trace)
    else:
        plot(trace)
        
        

def get_solution_stats(group_solution_array, agent_solution_array):
    
    solution_stats = {}
    
    # helper function to get number of unique elements in each column of an array
    def nunique_percol_sort(a):
        b = np.sort(a,axis=0)
        return (b[1:] != b[:-1]).sum(axis=0)+1
    
    solution_stats['time_intervals_working'] = nunique_percol_sort(group_solution_array).sum()
    
    group_job_reconfig_count = 0
    group_reconfig_count = 0
    for i in range(group_solution_array.shape[0]):
        for j in range(group_solution_array.shape[1]-1):
            if i == 0:
                if not np.array_equal(group_solution_array[:,j], group_solution_array[:,j+1]):
                    group_reconfig_count += 1
            if group_solution_array[i,j] != group_solution_array[i,j+1]:
                group_job_reconfig_count += 1
    
    solution_stats['group_job_reconfig_count'] = group_job_reconfig_count
    solution_stats['group_reconfig_count'] = group_reconfig_count
    
    agent_reconfig_count = 0
    for j in range(agent_solution_array.shape[1]-1):
        if not np.array_equal(agent_solution_array[:,j], agent_solution_array[:,j+1]):
            agent_reconfig_count += min(len(np.unique(agent_solution_array[:,j])), 
                                        len(np.unique(agent_solution_array[:,j+1])))
            
    solution_stats['agent_reconfig_count'] = agent_reconfig_count
    
    return solution_stats


### Generate a dummy problem

In this section we will generate some data for a sample problem, using functions from a previous notebook.

We first generate some taskload parameters. Here we are using: 

- 56 time intervals

- 14 sectors

- Seector taskload (Normal) distribution means generated by N~(100, 10), with their standard deviations a quarter of the mean

- A low level of zero valued taskloads

- Taskloads distributed to be higher at the beginning and end of the time horizon

In [3]:
num_time_intervals = 56
num_sectors = 14

taskload_parameters = generate_random_taskload_parameters(num_time_intervals, num_sectors, 
                                                          {'taskload_dist_means_mean' : 100, 
                                                           'taskload_dist_means_stdev' : 10,
                                                           'taskload_dist_stdev_ratio' : 0.25},
                                                          zero_taskload_density=0.25, 
                                                          time_horizon_profile='middle_heavy', 
                                                          random_state=1)
taskload_parameters

{'number_time_intervals': 56,
 'number_jobs': 14,
 'job_normal_dist_params_dict': {0: {'mean': 116.0, 'stdev': 29.0},
  1: {'mean': 94.0, 'stdev': 24.0},
  2: {'mean': 95.0, 'stdev': 24.0},
  3: {'mean': 89.0, 'stdev': 22.0},
  4: {'mean': 109.0, 'stdev': 27.0},
  5: {'mean': 77.0, 'stdev': 19.0},
  6: {'mean': 117.0, 'stdev': 29.0},
  7: {'mean': 92.0, 'stdev': 23.0},
  8: {'mean': 103.0, 'stdev': 26.0},
  9: {'mean': 98.0, 'stdev': 24.0},
  10: {'mean': 115.0, 'stdev': 29.0},
  11: {'mean': 79.0, 'stdev': 20.0},
  12: {'mean': 97.0, 'stdev': 24.0},
  13: {'mean': 96.0, 'stdev': 24.0}},
 'job_time_horizon_profiles_dict': {0: 'middle_heavy',
  1: 'middle_heavy',
  2: 'middle_heavy',
  3: 'middle_heavy',
  4: 'middle_heavy',
  5: 'middle_heavy',
  6: 'middle_heavy',
  7: 'middle_heavy',
  8: 'middle_heavy',
  9: 'middle_heavy',
  10: 'middle_heavy',
  11: 'middle_heavy',
  12: 'middle_heavy',
  13: 'middle_heavy'},
 'job_zero_taskload_probability_dict': {0: 0.028291534783718025,
  1: 0.

In [4]:
taskload_array = generate_taskload_array(taskload_parameters, random_state=0)
create_taskload_heatmap(taskload_parameters, taskload_array, in_notebook=True, 
                        colorscale='Greens',annotate_heatmap=False)

We also need to generate some valid configurations that the airspaces can take, and the associated airspaces.

In [5]:
def peturb_config_airspaces(config, peturb_index, peturb_direction=None):
    
    """
    Function to peturb an existing configuration of airspaces. Takes the airspace 
    (list of airspace indexes) at peturb index and moves one airspace to either the
    airspace on the right or left, as specified by the input. If the peturb index 
    is 0 then the airspace has to move to the right, and likewise if the index is
    the last one, the move has to be to the right. If the airspace at peturb index
    only has one elementary airspace (index) in it then we don't peturb anything 
    and just return the original config.
    """
    
    c = config.copy()
    
    if peturb_direction not in ['left','right']:
        #raise ValueError('The peturb direction can only be left or right')
        if np.random.rand() < 0.5:
            peturb_direction = 'right'
        else:
            peturb_direction = 'left'
    
    if peturb_index == 0:
        peturb_direction = 'right'
    if peturb_index == len(c) - 1:
        peturb_direction = 'left'
    
    # only peturb the config if the airspace being altered has more than one 
    # elementary sector in it
    if len(c[peturb_index]) > 1:
        if peturb_direction == 'right':
            new_config = c[:peturb_index] + [c[peturb_index][:-1]] + \
                [[c[peturb_index][-1]] + c[peturb_index+1]] + c[peturb_index+2:]
        elif peturb_direction == 'left':
            new_config = c[:peturb_index-1] + [c[peturb_index-1] + [c[peturb_index][0]]] + \
                [c[peturb_index][1:]] + c[peturb_index+1:]
        else:
            pass
        return new_config
    else:
        return c

In [6]:
peturb_config_airspaces([list(l) for l in np.array_split(np.arange(10), 4)], 2, 'right')

[[0, 1, 2], [3, 4, 5], [6], [7, 8, 9]]

In [40]:
def generate_airspaces_and_configurations(num_elementary_sectors, max_groups, num_configs_per_group_size):
    
    """
    Function to generate a set of configurations and the airspaces populating those configurations.
    Will use the config peturb function to give different but reasonably balanced configs, from
    which all the distinct airspaces will be taken.
    Returns a dictionary of cinfigurations to airspaces, airspaces to elementary sectors, and airspaces to
    configurations containing those airspaces.
    """
    
    elementary_sector_list = list(np.arange(num_elementary_sectors))
       
    # put in one sector which covers the whole airspace to start, as the first configuration
    airspaces_dict = {0: elementary_sector_list}
    configuration_dict = {0: [0]}
    airspaces_to_config_dict = {}
    
    # iterate through the number of groups and create new configs, then select 
    # num_configs_per_group_size configs to keep
    all_configs = []
    for num_groups in range(2, max_groups+1):
        # split groups as evenly as possible
        first_config = np.array_split(np.arange(num_elementary_sectors), num_groups)
        first_config = [list(l) for l in first_config]
        new_config_list = [first_config]
        for group_index in range(num_groups):
            if group_index in [0, num_groups-1]:
                new_config = peturb_config_airspaces(first_config, group_index)
                if new_config not in new_config_list:
                    new_config_list.append(new_config)
            else:
                new_config_left = peturb_config_airspaces(first_config, group_index, peturb_direction='left')
                new_config_right = peturb_config_airspaces(first_config, group_index, peturb_direction='right')
                if new_config_left not in new_config_list:
                    new_config_list.append(new_config_left)
                if new_config_right not in new_config_list:
                    new_config_list.append(new_config_right)
        new_configs_indexes = np.concatenate((np.array([0]), 
                                              np.random.permutation(np.arange(1,len(new_config_list)))[:num_configs_per_group_size-1]))
        picked_configs = [new_config_list[i] for i in new_configs_indexes]
        all_configs.append(picked_configs)
        
    all_configs = [i for l in all_configs for i in l]
        
    # run through the configs and add the airspaces to the dict if needed, and add the airspaces into the configs
    counter_airspace = 1
    counter_config = 1
    for config in all_configs:
        config_airspaces_list = []
        for airspace in config:
            # if the airspace is not in the airspaces_dict then add it with
            # the counter as its index, otherwise pick the index up from the
            # existing dict, then add it to the config dict
            if airspace not in airspaces_dict.values():
                airspaces_dict[counter_airspace] = airspace
                config_airspaces_list.append(counter_airspace)
                counter_airspace += 1
            else:
                airspace_index = [k for k, v in airspaces_dict.items() if v == airspace][0]
                config_airspaces_list.append(airspace_index)     
                
        configuration_dict[counter_config] = config_airspaces_list
        counter_config += 1
        
     # populate the dictionary of 
    for config, airspace_list in configuration_dict.items():
        for airspace in airspace_list:
            if airspace not in airspaces_to_config_dict.keys():
                airspaces_to_config_dict[airspace] = [config]
            else:
                airspaces_to_config_dict[airspace].append(config)
        
    return configuration_dict, airspaces_dict, airspaces_to_config_dict

In [70]:
configuration_dict, airspaces_dict, airspaces_to_config_dict = generate_airspaces_and_configurations(num_sectors, 5, 8)

In [71]:
configuration_dict

{0: [0],
 1: [1, 2],
 2: [3, 4],
 3: [5, 6],
 4: [7, 8, 9],
 5: [10, 11, 9],
 6: [7, 12, 13],
 7: [3, 14, 9],
 8: [7, 15, 16],
 9: [10, 17, 18, 16],
 10: [10, 17, 19, 9],
 11: [7, 20, 18, 16],
 12: [10, 17, 21, 22],
 13: [10, 23, 24, 16],
 14: [25, 26, 18, 16],
 15: [10, 27, 28, 16],
 16: [25, 29, 30, 31, 22],
 17: [10, 32, 30, 31, 22],
 18: [25, 29, 30, 24, 16],
 19: [25, 33, 34, 31, 22],
 20: [25, 29, 35, 21, 22],
 21: [25, 29, 14, 36, 22],
 22: [25, 29, 30, 37, 38],
 23: [25, 39, 12, 31, 22]}

In [72]:
airspaces_dict

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
 1: [0, 1, 2, 3, 4, 5, 6],
 2: [7, 8, 9, 10, 11, 12, 13],
 3: [0, 1, 2, 3, 4, 5],
 4: [6, 7, 8, 9, 10, 11, 12, 13],
 5: [0, 1, 2, 3, 4, 5, 6, 7],
 6: [8, 9, 10, 11, 12, 13],
 7: [0, 1, 2, 3, 4],
 8: [5, 6, 7, 8, 9],
 9: [10, 11, 12, 13],
 10: [0, 1, 2, 3],
 11: [4, 5, 6, 7, 8, 9],
 12: [5, 6, 7, 8],
 13: [9, 10, 11, 12, 13],
 14: [6, 7, 8, 9],
 15: [5, 6, 7, 8, 9, 10],
 16: [11, 12, 13],
 17: [4, 5, 6, 7],
 18: [8, 9, 10],
 19: [8, 9],
 20: [5, 6, 7],
 21: [8, 9, 10, 11],
 22: [12, 13],
 23: [4, 5, 6, 7, 8],
 24: [9, 10],
 25: [0, 1, 2],
 26: [3, 4, 5, 6, 7],
 27: [4, 5, 6],
 28: [7, 8, 9, 10],
 29: [3, 4, 5],
 30: [6, 7, 8],
 31: [9, 10, 11],
 32: [4, 5],
 33: [3, 4, 5, 6],
 34: [7, 8],
 35: [6, 7],
 36: [10, 11],
 37: [9, 10, 11, 12],
 38: [13],
 39: [3, 4]}

In [73]:
airspaces_to_config_dict

{0: [0],
 1: [1],
 2: [1],
 3: [2, 7],
 4: [2],
 5: [3],
 6: [3],
 7: [4, 6, 8, 11],
 8: [4],
 9: [4, 5, 7, 10],
 10: [5, 9, 10, 12, 13, 15, 17],
 11: [5],
 12: [6, 23],
 13: [6],
 14: [7, 21],
 15: [8],
 16: [8, 9, 11, 13, 14, 15, 18],
 17: [9, 10, 12],
 18: [9, 11, 14],
 19: [10],
 20: [11],
 21: [12, 20],
 22: [12, 16, 17, 19, 20, 21, 23],
 23: [13],
 24: [13, 18],
 25: [14, 16, 18, 19, 20, 21, 22, 23],
 26: [14],
 27: [15],
 28: [15],
 29: [16, 18, 20, 21, 22],
 30: [16, 17, 18, 22],
 31: [16, 17, 19, 23],
 32: [17],
 33: [19],
 34: [19],
 35: [20],
 36: [21],
 37: [22],
 38: [22],
 39: [23]}

In [88]:
def create_airspace_taskloads_dict(airspaces_dict, taskload_array):
    
    """
    Create dictionary of the taskload for each airspace in each time interval.
    """
    
    airspace_taskloads_dict = {}
    
    for airspace_index, airspace in airspaces_dict.items():
        for ti in range(num_time_intervals):
            airspace_taskloads_dict[(airspace_index, ti)] = sum([taskload_array[elementary_sector, ti] 
                                                                 for elementary_sector in airspace])
            
    return airspace_taskloads_dict

In [87]:
airspace_taskloads_dict = create_airspace_taskloads_dict(airspaces_dict, taskload_array)
airspace_taskloads_dict

{(0, 0): 1089,
 (0, 1): 1067,
 (0, 2): 924,
 (0, 3): 1093,
 (0, 4): 1032,
 (0, 5): 1092,
 (0, 6): 1144,
 (0, 7): 883,
 (0, 8): 1144,
 (0, 9): 1213,
 (0, 10): 1374,
 (0, 11): 1088,
 (0, 12): 1382,
 (0, 13): 1339,
 (0, 14): 1380,
 (0, 15): 1425,
 (0, 16): 1541,
 (0, 17): 1346,
 (0, 18): 1429,
 (0, 19): 1574,
 (0, 20): 1659,
 (0, 21): 1669,
 (0, 22): 1462,
 (0, 23): 1877,
 (0, 24): 1661,
 (0, 25): 1768,
 (0, 26): 1710,
 (0, 27): 1865,
 (0, 28): 1762,
 (0, 29): 1575,
 (0, 30): 1700,
 (0, 31): 1655,
 (0, 32): 1697,
 (0, 33): 1856,
 (0, 34): 1755,
 (0, 35): 1584,
 (0, 36): 1583,
 (0, 37): 1677,
 (0, 38): 1720,
 (0, 39): 1367,
 (0, 40): 1722,
 (0, 41): 1537,
 (0, 42): 1655,
 (0, 43): 1499,
 (0, 44): 1092,
 (0, 45): 1330,
 (0, 46): 1236,
 (0, 47): 1289,
 (0, 48): 1229,
 (0, 49): 1230,
 (0, 50): 1310,
 (0, 51): 1162,
 (0, 52): 956,
 (0, 53): 922,
 (0, 54): 941,
 (0, 55): 1146,
 (1, 0): 481,
 (1, 1): 481,
 (1, 2): 546,
 (1, 3): 556,
 (1, 4): 538,
 (1, 5): 581,
 (1, 6): 644,
 (1, 7): 370,
 (1, 8)

Write a model - at first ignore configurations and work solely with airspaces

In [60]:
def create_airspace_variables(model, airspaces_dict, num_time_intervals):
    
    """
    Create bool variables for an airspace being active in time period t.
    """
    
    airspace_variables = {}
    for airspace in range(len(airspaces_dict)):
        for ti in range(num_time_intervals):
            airspace_variables[(airspace, ti)] = model.NewBoolVar('airspace_%i_time_interval_%i' % (airspace, ti))
            
    return airspace_variables

In [61]:
def create_configuration_variables(model, configuration_dict, num_time_intervals):
    
    """
    Create bool variables for an configuration being active in time period t.
    """
    
    config_variables = {}
    for config in range(len(configuration_dict)):
        for ti in range(num_time_intervals):
            config_variables[(config, ti)] = model.NewBoolVar('config_%i_time_interval_%i' % (config, ti))
            
    return config_variables

In [62]:
def create_airspace_active_variables(model, configuration_dict, num_time_intervals):
    
    """
    Create int variables for counting the number of active airspaces in time period t.
    """
    
    max_airspaces = max([len(i) for i in configuration_dict.values()])
    
    airspace_active_variables = {}
    for ti in range(num_time_intervals):
        airspace_active_variables[ti] = model.NewIntVar(0, max_airspaces, 'time_interval_%i' % ti)
            
    return airspace_active_variables

In [63]:
def create_airspace_to_configuration_link_constraints(model, airspace_variables, config_variables,
                                                      airspaces_dict, airspaces_to_config_dict, 
                                                      num_time_intervals):
     
    """
    Create constraint to link the airspace variables to the configuration variables.
    """
    
    for airspace in range(len(airspaces_dict)):
        for ti in range(num_time_intervals):
            model.Add(airspace_variables[(airspace, ti)] == cp_model.LinearExpr.Sum(config_variables[(config, ti)]
                                                                                    for config in airspaces_to_config_dict[airspace]))
            
    return model

In [64]:
def create_configuration_active_constraints(model, config_variables, configuration_dict, num_time_intervals):
    
    """
    Create constraint to ensure that only one configuration is active at any one time.
    """
    
    for ti in range(num_time_intervals):
        model.Add(cp_model.LinearExpr.Sum(config_variables[(config, ti)] 
                                          for config in configuration_dict.keys()) == 1)
        
    return model

In [89]:
def create_taskload_limit_constraint(model, airspace_variables, airspaces_dict, airspace_taskloads_dict, 
                                     taskload_limit, num_time_intervals):
    
    """
    Create constraint to ensure that no airspace is active if its taskload exceeds 
    the taskload limit.
    """
    
    for airspace in range(len(airspaces_dict)):
        for ti in range(num_time_intervals):
            model.Add(airspace_variables[(airspace, ti)] * airspace_taskloads_dict[(airspace, ti)] <= taskload_limit)
            
    return model

In [66]:
def create_objective(model, airspace_active_variables, configuration_dict, num_time_intervals):
    
    max_airspaces = max([len(i) for i in configuration_dict.values()])
    
    # create objective value variable
    obj_val_var = model.NewIntVar(0, max_airspaces*num_time_intervals, 'airspace_open_count')
    
    # add constraint to set the variable
    model.Add(obj_val_var == sum([airspace_active_variables[ti] for ti in range(num_time_intervals)]))
    
    model.Minimize(obj_val_var)
    
    return model, obj_val_var

In [91]:
def create_model(configuration_dict, airspaces_dict, airspaces_to_config_dict, airspace_taskloads_dict, 
                 num_time_intervals, taskload_limit):
    
    # create model
    model = cp_model.CpModel()
    
    # create variables
    airspace_variables = create_airspace_variables(model, airspaces_dict, num_time_intervals)
    config_variables = create_configuration_variables(model, configuration_dict, num_time_intervals)
    airspace_active_variables = create_airspace_active_variables(model, configuration_dict, num_time_intervals)
    
    # create constraints
    model = create_airspace_to_configuration_link_constraints(model, airspace_variables, config_variables,
                                                              airspaces_dict, airspaces_to_config_dict, 
                                                              num_time_intervals)
    model = create_configuration_active_constraints(model, config_variables, configuration_dict, num_time_intervals)
    model = create_taskload_limit_constraint(model, airspace_variables, airspaces_dict, airspace_taskloads_dict, 
                                             taskload_limit, num_time_intervals)
    
    # create objective
    model, obj_val_variable = create_objective(model, airspace_active_variables, configuration_dict, num_time_intervals)
    
    return model, airspace_variables, config_variables, airspace_active_variables, obj_val_variable

In [68]:
def solve_model(model, max_solve_time=10):
    
    # solve model
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = max_solve_time
    #solver.parameters.search_branching = cp_model.FIXED_SEARCH
    solver.parameters.num_search_workers = 16
    #solver.parameters.randomize_search = True
    status = solver.Solve(model)
    
    return model, solver, status

In [ ]:
### PRINT RESULTS, FIGURE OUT HOW TO VISUALISE SOLUTIONS WITH NEW OUTPUTS ETC

def print_results(model, config_variables, obj_val_variable, solver):
    
    max_obj = 0
    
    for index_tuple, config_var in config_variables.items():
        conf, ti = index_tuple[0], index_tuple[1]
        val = solver.Value(config_var)
        #max_obj = max(max_obj, val)
        print('Config ' + str(node_index) + ': ' + str(val))
        
    print('Objective: ' + str(solver.Value(obj_val_variable)))

Create and solve the model.

In [58]:
taskload_limit = 600

In [93]:
modl, as_vars, conf_vars, asa_vars, obj_var = create_model(configuration_dict, airspaces_dict, 
                                                           airspaces_to_config_dict, airspace_taskloads_dict,
                                                           num_time_intervals, taskload_limit)

In [95]:
print(ctime())
modl, solv, stat = solve_model(modl, max_solve_time=20)
print(ctime())

Tue Jan 21 17:03:07 2020
Tue Jan 21 17:03:07 2020
